In [ ]:
"""
Daniel Wu
CSCI 544
Programming Assignment 1
"""

# import os
# os.getcwd()
# os.chdir('/Users/user/Desktop/Fall 2020/CSCI 544/coding-1-dev-data')

# remember to replace key with test

# remember to call this program full-name-predictor.py

# output the cleaned names as full-name-output.csv

import os
import csv

def pause():
    programPause = input("Press the <ENTER> key to continue...")
    print("Paused Program")

# Bring in Last Name List:
last_name_raw = []
with open('Names_2010Census.csv', 'r') as lastname_file:
    lastname_reader = csv.reader(lastname_file)
    next(lastname_reader)        
    for lastname in lastname_reader:                        
        last_name_raw.append(lastname)
        

i = 0
for i in range(len(last_name_raw)):
    del last_name_raw[i][4:]
    del last_name_raw[i][2]
    last_name_raw[i][2] = float(last_name_raw[i][2]) / 100000
    last_name_raw[i][2], last_name_raw[i][1] = last_name_raw[i][1], last_name_raw[i][2]
        
# Bring in First Name Male List:
malename_raw = ""
with open('dist.male.first.txt', 'r') as malename_file:
    malename_raw = malename_file.read()
    malename_raw = malename_raw.split('\n')
    del malename_raw[-1]

for i in range(len(malename_raw)):
    malename_raw[i] = ' '.join(malename_raw[i].split())
    malename_raw[i] = malename_raw[i].split()
    del malename_raw[i][2]
    malename_raw[i][1] = float(malename_raw[i][1])
    malename_raw[i][2] = float(malename_raw[i][2])

# Bring in First Name Female List:
femalename_raw = ""
with open('dist.female.first.txt', 'r') as femalename_file:
    femalename_raw = femalename_file.read()
    femalename_raw = femalename_raw.split('\n')
    del femalename_raw[-1]

for i in range(len(femalename_raw)):
    femalename_raw[i] = ' '.join(femalename_raw[i].split())
    femalename_raw[i] = femalename_raw[i].split()
    del femalename_raw[i][2]
    femalename_raw[i][1] = float(femalename_raw[i][1])
    femalename_raw[i][2] = float(femalename_raw[i][2])  

    
def splitlists(census_data):
    
    names = []
    rank = []
    stats = []
    
    for each_list in census_data:
        names.append(each_list[0])
        stats.append(each_list[1])
        rank.append(each_list[2])        
    return names, stats, rank

census_malenames, census_malename_stats, census_malename_rank = splitlists(malename_raw)
census_femalenames, census_femalename_stats, census_femalename_rank = splitlists(femalename_raw)
census_lastnames, census_lastnames_stats, census_lastnames_rank = splitlists(last_name_raw)
    
RawNames = []            # Read from CSV
Name_Split = []          # Split by " AND "
First_Person_Name = []   # Uncleaned first person name (To Update)
Second_Person_Name = []   # Uncleaned second person name (To Work With but not update)

with open('dev-test.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)    
    for names in csv_reader:        
        RawNames.append(names)
                
for i in range(len(RawNames)):
    
    # Create First and Second person's full name
    name_iter = RawNames[i][0].split(" AND ", maxsplit = 1) 
    Name_Split.append(name_iter)            
    First_Person_Name.append(name_iter[0]) 
    Second_Person_Name.append(name_iter[1]) 
        
    First_Person_Words = First_Person_Name[i].split(" ")
    Second_Person_Words = Second_Person_Name[i].split(" ")
    
    first_len = len(First_Person_Words) - (First_Person_Words[0].count('PROFESSOR') + 
                                           First_Person_Words[0].count('COLONEL') + 
                                           First_Person_Words[0].count('REVEREND') +
                                           First_Person_Words[0].count('DOCTOR') +
                                           First_Person_Words[0].count('MAJOR'))
    
    second_len = len(Second_Person_Words) - (Second_Person_Words[0].count('PROFESSOR') + 
                                             Second_Person_Words[0].count('COLONEL') + 
                                             Second_Person_Words[0].count('REVEREND') + 
                                             First_Person_Words[0].count('DOCTOR') +
                                             Second_Person_Words[0].count('MAJOR'))    
 
    if first_len <= second_len and first_len <= 2:
        
        # do this unconditionally since it seems to perform well
        First_Person_Name[i] = str(First_Person_Name[i]) + " " + str(Second_Person_Words[-1])
                
        # For tweaks, be extremely conservative (i.e only change the status quo if almost certain its wrong)
        
        first_check_last_1 = census_lastnames.count(First_Person_Words[-1])
        first_check_male_fore_1 = census_malenames.count(First_Person_Words[-1])            
        first_check_female_fore_1 = census_femalenames.count(First_Person_Words[-1])            
        
        # now, use bayesian probability to test whether its more likely to be a surname or forename 
                
        if first_check_last_1 == 1:            
            last_index = census_lastnames.index(First_Person_Words[-1])            
            joint_lastname_prob = census_lastnames_stats[last_index]            
        elif first_check_last_1 == 0:
            joint_lastname_prob = 0
            
        if first_check_male_fore_1 == 1:
            male_index = census_malenames.index(First_Person_Words[-1])            
            joint_malename_prob = census_malename_stats[male_index]
        elif first_check_male_fore_1 == 0:
            joint_malename_prob = 0
        
        if first_check_female_fore_1 == 1:
            female_index = census_femalenames.index(First_Person_Words[-1])            
            joint_femalename_prob = census_femalename_stats[female_index]
        elif first_check_female_fore_1 == 0:
            joint_femalename_prob = 0
        
        if joint_lastname_prob != 0 or joint_malename_prob != 0 or joint_femalename_prob != 0:
            pr_last = joint_lastname_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
            pr_male = joint_malename_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
            pr_female = joint_femalename_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
        else:
            pr_last = 0
            pr_male = 0
            pr_female = 0

        if pr_last > pr_male and pr_last > pr_female and pr_last != 0:
        
            First_Person_Name[i] = ' '.join(First_Person_Name[i].split(' ')[:-1])
        
        
        
        check_last_2 = census_lastnames.count(Second_Person_Words[-2])            
        check_male_fore_2 = census_malenames.count(Second_Person_Words[-2])            
        check_female_fore_2 = census_femalenames.count(Second_Person_Words[-2])        
                
        # Use bayesian probability again for the second guys second last word
        
        if check_last_2 == 1:            
            last_index = census_lastnames.index(Second_Person_Words[-2])            
            joint_lastname_prob = census_lastnames_stats[last_index]            
        elif check_last_2 == 0:
            joint_lastname_prob = 0
            
        if check_male_fore_2 == 1:
            male_index = census_malenames.index(Second_Person_Words[-2])            
            joint_malename_prob = census_malename_stats[male_index]
        elif check_male_fore_2 == 0:
            joint_malename_prob = 0
        
        if check_female_fore_2 == 1:
            female_index = census_femalenames.index(Second_Person_Words[-2])            
            joint_femalename_prob = census_femalename_stats[female_index]
        elif check_female_fore_2 == 0:
            joint_femalename_prob = 0
        
        if joint_lastname_prob != 0 or joint_malename_prob != 0 or joint_femalename_prob != 0:
            pr_last = joint_lastname_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
            pr_male = joint_malename_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
            pr_female = joint_femalename_prob / (joint_lastname_prob + joint_malename_prob + joint_femalename_prob)
        else:
            pr_last = 0
            pr_male = 0
            pr_female = 0

        if pr_last > pr_male and pr_last > pr_female and pr_last != 0:
    
            temp_name = First_Person_Name[i].split(' ')
            temp_name.insert(-1, Second_Person_Words[-2])
            First_Person_Name[i] = ' '.join(temp_name)
            
CleanedNames = RawNames

for i in range(len(CleanedNames)):        
    RawNames[i].append(First_Person_Name[i])
    
with open('full-name-output.csv', 'w') as new_file:        
    csv_writer = csv.writer(new_file)

    for line in CleanedNames:
        csv_writer.writerow(line)

In [9]:
RawNames = []

with open('/Users/user/Desktop/Fall 2020/CSCI 544/Coding Assignments/PA1/coding-1-dev-data/dev-test.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)    
    for names in csv_reader:        
        RawNames.append(names)